In [ ]:
# basuc library
import numpy as np
import pandas as pd

# #import my kaggle_utiles file that has all the custom funcitons i want.
# import sys
# sys.path.append("/home/pavithra/Pictures/learning/ML/kaggle/")
# sys.path
import kaggle_utils_py as kaggle_utils


# data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# evaluation metrics
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
pd.set_option('display.max_columns', None)

# set the warning off
import warnings
warnings.filterwarnings("ignore")

# Index
- [1. Describtive analysis (Univariant Analysis)](#1)
    - [1.1 Common Data](#1.1)
        - [1.1.1 Data type and Missing values Analysis](#1.1.1)
        - [1.1.2 Zero variance Features](#1.1.2)
    - [1.2 Numerical Data ](#1.2)
        - [1.2.1 Quantile statistics](#1.2)
        - [1.2.2 Descriptive analysis](#1.2)
        - [1.2.3 Distribution Analysis](#1.2)
            - [1.2.3.2 Outlier Detection](#1.2.3.2)
    - [1.3 Categorical Data](#1.3)
        - [1.3.1 Cordinality/ Unnique count ](#1.3.1)
        - [1.3.2 Target Class balance check(only for classification)](#1.3.3)
- [2. Correlation Analysis (Bivariant Analysis)](#2)
- [3. Data Cleaning](#3)
- [4. Hyper parameter tuning](#4)
- [5. Modeling](#5)

In [ ]:
# read the data
data = pd.read_csv("../input/tabular-playground-series-may-2022/train.csv", index_col=0)
test = pd.read_csv("../input/tabular-playground-series-may-2022/test.csv", index_col=0)
sub = pd.read_csv("../input/tabular-playground-series-may-2022/sample_submission.csv")

data.head()

In [ ]:
test.head()

In [ ]:
print("shape of the data --->", data.shape)
print("shape of the test data --->", test.shape)

<a id="1"></a>
# <p style="background-color:#5811D3;font-family:newtimeroman;color:#FEDFA0;font-size:120%;text-align:center;border-radius:10px 10px;">Descriptive Analysis</p>

<a id="1.1"></a>
# <p style="background-color:#5811D3;font-family:newtimeroman;color:#FEDFA0;font-size:100%;text-align:left;border-radius:10px 10px;">1.1 | Common data Analysis</p>

<a id="1.1.1"></a>
# <p style="font-family:newtimeroman;color:#5811D3;font-size:100%;text-align:left;border-radius:10px 10px;">1.1.1 | Data type and Missing values Analysis</p>

In [ ]:
columns, categorical_col, numerical_col,missing_value_df = kaggle_utils.Common_data_analysis(data, missing_value_highlight_threshold=5.0, display_df = True,
                                                                                         only_show_missing=False)

In [ ]:
target = "target"
numerical_col.remove(target)

## observation
- All features are numberical columns except f_27
- No missing values :) 

<a id="1.1.2"></a>
# <p style="font-family:newtimeroman;color:#5811D3;font-size:100%;text-align:left;border-radius:10px 10px;">1.1.2 | Zero variance Features</p>

In [ ]:
zero_var_col = []
for col in data.columns:
    if len(data[col].unique()) == 1:
        print(data[col].unique(), end= "")
        zero_var_col.append(col)
print()
print(zero_var_col, end="")
print()
print("Total number of features having zero variance -->", len(zero_var_col))

<a id="1.2"></a>
# <p style="background-color:#5811D3;font-family:newtimeroman;color:#FEDFA0;font-size:100%;text-align:left;border-radius:10px 10px;">1.2 | Numerical Data -- descriptive, distribution, Quantitative</p>

In [ ]:
kaggle_utils.numerical_data_analysis(data[numerical_col], numerical_col)

## observation
- All features are within some range, example f_00 - f_06 are with the range of [-5, +5] -- continuous features
- f_07 - f_18 seems to be a descrete numerical features with some range -- mostly 0 - 14 (may be a ordinal value)
- Again f_19 - f_28 are within some range -- continuous values
- f_29 and f_30 are ordinal values

- **target** values has median zero -- so we have more than 50% of data is zero (may be a balanced dataset)

<a id="1.2.3.2"></a>
# <p style="font-family:newtimeroman;color:#5811D3;font-size:100%;text-align:left;border-radius:10px 10px;">1.2.3.2 | Outlier detection</p>

In [ ]:
outlier_df = kaggle_utils.find_outliers_iqr_method(data)
print("Number of feature having outliers ---> ",len(outlier_df[outlier_df["Number of outliers"] > 0]))
outlier_df[outlier_df["Number of outliers"] > 0].T

In [ ]:
def plot_hist(data, columns, nrow, ncol, figsize, hue_value=None):
    # find the distubution of the data. ( visualization would be so good)
    fig, ax = plt.subplots(nrow,ncol, figsize=figsize)
    # we have 9 numerical values.
    col, row = ncol,nrow
    col_count = 0
    if row<=1:
        for c in range(col):
            if hue_value:
                sns.histplot(data=data, x=columns[col_count],kde=True, hue=hue_value, ax=ax[c])
            else:
                sns.histplot(data=data, x=columns[col_count], kde=True, ax=ax[c])
    else:
        for r in range(row):
            for c in range(col):
                if col_count >= len(columns):
                    ax[r,c].text(0.5, 0.5, "no data")
                else:
                    if hue_value:
                        sns.histplot(data=data, x=columns[col_count],kde=True, hue=hue_value, ax=ax[r,c])
                    else:
                        sns.histplot(data=data, x=columns[col_count],kde=True,  ax=ax[r,c])
                col_count +=1
def plot_boxplot(data, columns, nrow, ncol, figsize, hue_value=None):
    # find the distubution of the data. ( visualization would be so good)
    fig, ax = plt.subplots(nrow,ncol, figsize=figsize)
    # we have 9 numerical values.
    col, row = ncol,nrow
    col_count = 0

    for r in range(row):
        for c in range(col):
            if col_count >= len(columns):
                ax[r,c].text(0.5, 0.5, "no data")
            else:
                if hue_value:
                    sns.boxplot(data=data, x=columns[col_count],hue=hue_value, ax=ax[r,c])
                else:
                    sns.boxplot(data=data, x=columns[col_count], ax=ax[r,c])
            col_count +=1

def plot_scatter(data,common_y, columns, nrow, ncol, figsize):
    # find the distubution of the data. ( visualization would be so good)
    fig, ax = plt.subplots(nrow,ncol, figsize=figsize)
    # we have 9 numerical values.
    col, row = ncol,nrow
    col_count = 0

    for r in range(row):
        for c in range(col):
            if col_count >= len(columns):
                ax[r,c].text(0.5, 0.5, "no data")
            else:
                
                sns.scatterplot(data=data, x=columns[col_count],y=common_y, ax=ax[r,c])
            col_count +=1


In [ ]:
# plot the histogram
plot_hist(data, numerical_col, 5,6, (30,30))

In [ ]:
plot_boxplot(data, numerical_col, 5, 6, figsize=(30,30))

## observation
- All cotinuoue features has very decent normal distribution centered zero. Having few outliers (will decide after the base model about outlier)
- All Descrete features have large number of values in starting categories (say 1 -5) and very few data in other categories.

<a id="1.3"></a>
# <p style="background-color:#5811D3;font-family:newtimeroman;color:#FEDFA0;font-size:100%;text-align:left;border-radius:10px 10px;">1.3 | Categorical Data Analysis</p>

In [ ]:
# number of categorical features
print("Categorical col --> ", categorical_col )

<a id="1.3.1"></a>
# <p style="font-family:newtimeroman;color:#5811D3;font-size:100%;text-align:left;border-radius:10px 10px;">1.3.1 | Cordinality/ Unnique count</p>

In [ ]:
data[categorical_col[0]].value_counts()

In [ ]:
nunique = data[categorical_col[0]].nunique()
data_len = data.shape[0]

print("we have {} unique values in our dataset having {} samples ----> {}% different categories".format(nunique, data_len, (nunique/data_len)*100))

## observation 
- We can't use onhot encoding --  we can't use ordinal encoding -- Fequency encoding is also not a better idea since most of the categories having single sample.
- Seems like each values has some pattern, we can make use of that pattern to generate some meaningful feature.

<a id="1.3.2"></a>
# <p style="font-family:newtimeroman;color:#5811D3;font-size:100%;text-align:left;border-radius:10px 10px;">1.3.2 | Target Class balance check(only for classification)</p>

In [ ]:
# mostly has equal number of samples in both classes
sns.countplot(data=data, y=target)

In [ ]:
count = data[target].value_counts()
print(count)
print("percentage of first class --- >",count[0]/data.shape[0])
print("percentage of second class --->", count[1]/data.shape[0])

<a id="2"></a>
# <p style="background-color:#5811D3;font-family:newtimeroman;color:#FEDFA0;font-size:100%;text-align:left;border-radius:10px 10px;">2 ) Correlation Analysis (Bivariant Analysis)</p>

# <p style="font-family:newtimeroman;color:#5811D3;font-size:100%;text-align:left;border-radius:10px 10px;"> 2.1 | Corrilation between variables</p>

In [ ]:
plt.figure(figsize=(25,8))
sns.heatmap(data.corr(), annot=True, cbar=True, cmap="YlGnBu")

In [ ]:
corr_df = data.corr()

In [ ]:
def find_high_correlation(corr_df, threshold):
    col_corr_dict = {}
    for row in range(len(corr_df.columns)):
        for col in range(row): # the upper half and lower half is same -- so checking to half way is enough
            if corr_df.iloc[row, col] >= threshold or corr_df.iloc[row, col] <= -threshold:
                #print(corr_df.columns[col], corr_df.index[row])
                col_corr_dict[corr_df.index[row]] = [corr_df.columns[col], corr_df.iloc[row, col]] # these two values are highly correlated
    return col_corr_dict

threshold = 0.40
col_corr_dict = find_high_correlation(corr_df, threshold)

print(col_corr_dict)

In [ ]:
corr_df

# <p style="font-family:newtimeroman;color:#5811D3;font-size:100%;text-align:left;border-radius:10px 10px;"> 2.2 | correlation with target</p>

In [ ]:
corr_with_target = corr_df["target"]
corr_with_target.to_frame().T

In [ ]:
corr_with_target = corr_with_target[:-1]

In [ ]:
# distribution of correlation
# distirbution of the correlation with the target value
plt.title("Correlation distribution between the variables and target")
plt.xlabel("Correlation with target")
plt.ylabel("Number of features")
sns.histplot(corr_with_target)

# observation | from whole EDA
- **No null** values :)
- **No zero variance** feature :)
- **One categorical** value -- Need some custom handling method for this.
- **Not so much correlated** features :)
- All features are **almost normally** distributed :)
- All features has **outliers** :( --> but since they are normally distributed we can ignore this for now.

- No datetime features :)

<a id="3"></a>
# <p style="background-color:#5811D3;font-family:newtimeroman;color:#FEDFA0;font-size:100%;text-align:left;border-radius:10px 10px;"> 3 ) Data Cleaning</p>

<a id="1.3.1"></a>
# <p style="font-family:newtimeroman;color:#5811D3;font-size:100%;text-align:left;border-radius:10px 10px;">Decoding f_27</p>

In [ ]:
data["f_27"] # this is a length 10 string 

In [ ]:
# distibution of letter in this feature --  from "Exploring the string feature notebook"
unique_letters = set(())
from collections import Counter
c = Counter()
cps = [Counter() for _ in range(10)]
for i,astr in enumerate(data['f_27']):
   
    letters = [str(x) for x in astr]
    unique_letters.update(letters)
    c.update(letters)
    for i in range(10):
        cps[i].update([letters[i],])
    if i == 10**12:
        break

letter_freqs = pd.DataFrame(c.items())
letter_freqs.columns = ['letter', 'count']
_=sns.barplot(data=letter_freqs.sort_values(by=['letter']), x='letter', y='count')

This has values from A - T . what we are going to decide this 
- Going to crete 10 features representing which letter present in each position 
- Gonna add 'unique_feature_count' -- which will contain the unhique features in that string
- Gonna add 'sum_of_string" -- will contain the summ of all letter in the string

In [ ]:
for i in range(10):
    data["position_"+ str(i)] = data.f_27.str.get(i).apply(ord) - ord('A') + 1
    test["position_"+ str(i)] = test.f_27.str.get(i).apply(ord) - ord('A') + 1
# add unique_feature_count -- this feature from https://www.kaggle.com/code/cabaxiom/tps-may-22-eda-lgbm-model
data["unique_feature_count"] = data.f_27.apply(lambda s: len(set(s)))
test["unique_feature_count"] = test.f_27.apply(lambda s: len(set(s)))

# # add sum_of_string
data["sum_of_string"] = data.iloc[:,32:42].sum(axis=1)
test["sum_of_string"] = test.iloc[:,32:42].sum(axis=1)
data.head()

In [ ]:
data.drop("f_27", axis= 1, inplace=True)
test.drop("f_27", axis =1, inplace=True)

<a id="1.3.1"></a>
# <p style="font-family:newtimeroman;color:#5811D3;font-size:100%;text-align:left;border-radius:10px 10px;"> Scaling data</p>

In [ ]:
data1 = data.copy()

In [ ]:
data = data1.copy()

In [ ]:
data = data.drop("target", axis=1)

stand_scal = StandardScaler()
stand_scal.fit(data)
data = pd.DataFrame(stand_scal.transform(data), columns=data.columns)
test = pd.DataFrame(stand_scal.transform(test), columns=test.columns)

In [ ]:
data

<a id="4"></a>
# <p style="background-color:#5811D3;font-family:newtimeroman;color:#FEDFA0;font-size:100%;text-align:left;border-radius:10px 10px;"> 3 | Base models</p>

In [ ]:
trainable_features = data.columns.to_list()
#trainable_features.remove("target")

data_features = data[trainable_features].copy()
target_feature = data1["target"].copy()
X_train, X_val, y_train, y_val = train_test_split(data_features, target_feature, train_size=0.2, random_state=24)

In [ ]:
# logestic regression
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)
y_pred = log_reg.predict(X_val)
print("Accuracy --->", accuracy_score(y_true=y_val,y_pred=y_pred))

In [ ]:

ran_forest = RandomForestClassifier()
ran_forest.fit(X_train,y_train)
y_pred = ran_forest.predict(X_val)
print("Accuracy --->", accuracy_score(y_true=y_val,y_pred=y_pred))

In [ ]:
# # running more than one hour without any results :( -- leave this algo for this dataset
# svc = SVC()
# svc.fit(X_train,y_train)
# y_pred = svc.predict(X_val)
# print("Accuracy --->", accuracy_score(y_true=y_val,y_pred=y_pred))

In [ ]:

# knn = KNeighborsClassifier()
# knn.fit(X_train,y_train)
# y_pred = knn.predict(X_val)
# print("Accuracy --->", accuracy_score(y_true=y_val,y_pred=y_pred))

# Logestic regression -- Checking Assumption, Why it is not a best model here 

## Assumptions of logestic regerssion
- 1. Appropriate Outcome Type
- 2. Sufficiently large sample size
- 3. Absence of multicollinearity
- 4. No strongly influential outliers
- 5. Linearity of independent variables and log-odds
- 6. Independent of Abservation

---

### ASSUMPTION 1 -- you should have a binary classification problem -- logostic regression only works with binary class problems 
- THIS PROBLEM -- ASSUMPTION 1 --- SATISFIED :) :)

In [ ]:

data1[target].nunique()

---

### ASSUMPTION 2
- evry categorical feature must have >=10 samples per each target class  or   have atleast 500 samples in total

- THIS PROBLEM -- ASSUMPTION 2 --- SATISFIED :) :)

In [ ]:

print(data.shape[0])
for i in data.columns.to_list():
    if data.dtypes[i] == 'uint8': # keep only col values
        data[i].value_counts()

---

### ASSUMPTION 3 -- Absence of multicollinearity
- there should not be any features has correlated with any other features (no two/three / more featres linearly correlated)
- can be found by correlation matrix -- Correlation matrix can be difficult to interpret when there are many independent variables
- Furthermore, not all collinearity problems can be detected by inspection of the correlation matrix: it is possible for collinearity to exist between three or more variables even if no pair of variables has a particularly high correlation.

- VIF Variance Inflation Factor can be used
(Include constant in VIF calculation. Reference: https://stackoverflow.com/questions/42658379/variance-inflation-factor-in-python)

- VIF
    - The threshold for VIF is usually 5 (i.e. values above 5 means there is presence of multicollinearity)
    - Since all the variables have VIF <5, it means that there is no multicollinearity, and this assumption is satisfied
    - Let's have a look at the situation where we did not drop the first variable upon getting dummies:


In [ ]:

from statsmodels.stats.outliers_influence import variance_inflation_factor
# Use variance inflation factor to identify any significant multi-collinearity
def calc_vif(df):
    vif = pd.DataFrame()
    vif["variables"] = df.columns
    vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return(vif)

calc_vif(data)

- If there is perfect correlation, then VIF = infinity. This shows a perfect correlation between two independent variables. In the case of perfect correlation, we get R2 =1, which lead to 1/(1-R2) infinity. To solve this problem we need to drop one of the variables from the dataset which is causing this perfect multicollinearity.

In [ ]:
keep_col = [col for col in data.columns if col.startswith("f_")]
data_without_f_27 = data[keep_col]
data_without_f_27

In [ ]:
calc_vif(data_without_f_27)

In [ ]:
# try to fit the logestic regresion model here
data_features = data_without_f_27
target_feature = data1["target"].copy()
X_train, X_val, y_train, y_val = train_test_split(data_features, target_feature, train_size=0.2, random_state=24)


# logestic regression
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)
y_pred = log_reg.predict(X_val)
print("Accuracy --->", accuracy_score(y_true=y_val,y_pred=y_pred))

---

### ASSUMPTION  4  -- No strongly influential outliers
- Influential values are extreme individual data points that can alter the quality of the logistic regression model.
- Cook’s Distance is an estimate of the influence of a data point. It takes into account both the leverage and residual of each observation. Cook’s Distance is a summary of how much a regression model changes when the ith observation is removed.
- A general rule of thumb is that any observation with a Cook’s distance greater than 4/n (where n = total observations) is considered to be influential (https://www.statology.org/cooks-distance-python/ and https://www.scikit-yb.org/en/latest/api/regressor/influence.html?highlight=cook#module-yellowbrick.regressor.influence), though there are even more generic cutoff values of >0.5-1.0.

- For outliers, we can use the absolute standardized residuals to identify them (std resid > 3) ( which are all the data has standarnd deviation above +/- 3 is called outliers -- only applies for nor mal distribution)

#### 

In [ ]:
# from statsmodels.genmod.generalized_linear_model import GLM
# from statsmodels.genmod import families

# # Use GLM method for logreg here so that we can retrieve the influence measures
# logit_model = GLM(target_feature, data, family=families.Binomial())
# logit_results = logit_model.fit()
# print(logit_results.summary())


In [ ]:
# from scipy import stats

# # Get influence measures
# influence = logit_results.get_influence()

# # Obtain summary df of influence measures
# summ_df = influence.summary_frame()

# # Filter summary df to Cook distance
# diagnosis_df = summ_df.loc[:,['cooks_d']]

# # Append absolute standardized residual values
# diagnosis_df['std_resid'] = stats.zscore(logit_results.resid_pearson)
# diagnosis_df['std_resid'] = diagnosis_df.loc[:,'std_resid'].apply(lambda x: np.abs(x))

# # Sort by Cook's Distance
# diagnosis_df.sort_values("cooks_d", ascending=False)
# diagnosis_df

In [ ]:
# # Set Cook's distance threshold
# cook_threshold = 4 / len(X)
# print(f"Threshold for Cook Distance = {cook_threshold}")
# Plot influence measures (Cook's distance)
# fig = influence.plot_index(y_var="cooks", threshold=cook_threshold)
# plt.axhline(y=cook_threshold, ls="--", color='red')
# fig.tight_layout(pad=2)
# # Find number of observations that exceed Cook's distance threshold
# outliers = diagnosis_df[diagnosis_df['cooks_d'] > cook_threshold]
# prop_outliers = round(100*(len(outliers) / len(X)),1)
# print(f'Proportion of data points that are highly influential = {prop_outliers}%')
# # Find number of observations which are BOTH outlier (std dev > 3) and highly influential
# extreme = diagnosis_df[(diagnosis_df['cooks_d'] > cook_threshold) & 
#                        (diagnosis_df['std_resid'] > 3)]
# prop_extreme = round(100*(len(extreme) / len(X)),1)
# print(f'Proportion of highly influential outliers = {prop_extreme}%')





# remove the outliers or not is decided by the domain expert -- here we don't know what the features are first of all so we can't say a datapoint is a outlier or not

---

### ASSUMPTION  5  -- Linearity of independent variables and log odds
Box-Tidwell Test
- One of the important assumptions of logistic regression is the linearity of the logit over the continuous covariates. This assumption means that relationships between the continuous predictors and the logit (log odds) is linear.
- The Box-Tidwell transformation (test) can be used to test the linearity in the logit assumption when performing logistic regression.
- It checks whether the logit transform is a linear function of the predictor, effectively adding the non-linear transform of the original predictor as an interaction term to test if this addition made no better prediction.
- A statistically significant p-value of the interaction term in the Box-Tidwell transformation means that the linearity assumption is violated
- If one variable is indeed found to be non-linear, then we can resolve it by incorporating higher order polynomial terms for that variable in the regression analysis to capture the non-linearity (e.g. x^2) .- Another solution to this problem is the - categorization of the independent variables. That is transforming metric variables to ordinal level and then including them in the model.

- I am going to use visual check here . 
- that is plot between features and log(p / 1-p ) shold be linear -- p is probability of a sample being positve

In [ ]:
data_features = data_without_f_27
target_feature = data1["target"].copy()
#X_train, X_val, y_train, y_val = train_test_split(data_features, target_feature, train_size=0.2, random_state=24)


# logestic regression
log_reg = LogisticRegression()
log_reg.fit(data_features,target_feature)
y_pred = log_reg.predict_proba(data_features)


In [ ]:
y_pred # first element is probability of sample being +ve -- second is probability of sample being -ve


In [ ]:
# Get log odds values
log_odds = np.log(y_pred[:,0] / (1 - y_pred[:,0]))

# Visualize predictor continuous variable vs logit values (Age)
plot_scatter(data_without_f_27, log_odds, data_without_f_27.columns.to_list(), 5,6, figsize=(30,30))

- Can't say all the featres are linearly related to logodds -- better to tranform the data or  try some complex model 

---

### ASSUMPTIONS 6 -- Independence of observations
- Error terms need to be independent. That is that the data-points should not be from any dependent samples design, e.g., before-after measurements, or matched pairings

- Here the samples are indepentant -- so assumption satisfied

In [ ]:
# # can be check by
# # Setup logistic regression model (using GLM method so that we can retrieve residuals)
# logit_model = GLM(y, X_constant, family=families.Binomial())
# logit_results = logit_model.fit()
# print(logit_results.summary())
# # Generate residual series plot
# fig = plt.figure(figsize=(8,5))
# ax = fig.add_subplot(111, title="Residual Series Plot",
#                     xlabel="Index Number", ylabel="Deviance Residuals")

# # ax.plot(X.index.tolist(), stats.zscore(logit_results.resid_pearson))
# ax.plot(X.index.tolist(), stats.zscore(logit_results.resid_deviance))
# plt.axhline(y=0, ls="--", color='red');


# credits = https://towardsdatascience.com/assumptions-of-logistic-regression-clearly-explained-44d85a22b290
# https://github.com/kennethleungty/Logistic-Regression-Assumptions/blob/main/Logistic_Regression_Assumptions.ipynb

- Assumption 1 -- YES
- Assumption 2 -- YES
- Assumption 3 -- YES
- Assumption 4 -- NO
- Assumption 5 -- NO
- Assumption 6 -- YES

---

- Overfitting / Underfitting -- CURRECT FIT

In [ ]:
trainable_features = data.columns.to_list()
#trainable_features.remove("target")

data_features = data[trainable_features].copy()
target_feature = data1["target"].copy()
X_train, X_val, y_train, y_val = train_test_split(data_features, target_feature, train_size=0.2, random_state=24)
# logestic regression
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)
y_pred_train = log_reg.predict(X_train)
y_pred_val = log_reg.predict(X_val)
print("Accuracy for training data--->", accuracy_score(y_true=y_train,y_pred=y_pred_train))
print("Accuracy for validation data--->", accuracy_score(y_true=y_val,y_pred=y_pred_val))

---

### Try removing outliers


In [ ]:
continuous_features = [col for col in data.columns.to_list() if data[col].nunique() > 15]
print(continuous_features)

In [ ]:
# i am gonna use un scaled feature 
lower_limit , upper_limit = kaggle_utils.find_outlier_z_score_method(data1[continuous_features], return_limits=True)
lower_limit["upper_limit"] = upper_limit["upper limit"]
lower_limit

In [ ]:
print(data1.shape)

In [ ]:
lower_limit.loc[2, "lower limit"]

In [ ]:
drop_index = set()
for index,col in enumerate(continuous_features):
    for sample_index,sample in enumerate(data1[col]):
        if sample < lower_limit.loc[index, "lower limit"] or  sample > lower_limit.loc[index, "upper_limit"]:
            drop_index.add(sample_index)
drop_index

In [ ]:
len(drop_index)

In [ ]:
len(drop_index) / data1.shape[0] * 100 # i decided to loss this 8% of data

In [ ]:
# data2 = data1.copy()

In [ ]:
data1.drop(drop_index, axis=0, inplace=True)

In [ ]:
data1.shape

In [ ]:
# build the model with this data
data_features = data1
target_feature = data1["target"].copy()
data_features = data1.drop("target",axis=1)
X_train, X_val, y_train, y_val = train_test_split(data_features, target_feature, train_size=0.2, random_state=24)


# logestic regression
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)
y_pred = log_reg.predict(X_val)
print("Accuracy -->", accuracy_score(y_val, y_pred))

#### No use -- better to go with some complex model
### resons
- the target label has no linear correlation with the features
- Having  outliers
- others things are very hard to understand for me .. since i am starting this theory checklist now -- need to learn lot of statistical and mathematical terms to describe more


# SVM - loading ..... :)

# Conclusion
- Why i am trying some basic models? 
I know Deep learning/ XGBoost will give the top scores...... but, I wanna learn how all( alomost know) models working from this ..... 